# Project Discrete Optimization and Decision Making

## First point

This project involves designing an efficient last-mile delivery system for a city. A company has a set of customers $C = \{1, \ldots, \overline{c}\}$, each requiring the delivery of a package. The weight of each package $w_c, c \in C$ is known. The delivery time for each customer is equal to $s_c, c \in C$. The company has a set $K = \{1, \ldots, \overline{k}\}$ of delivery vans available that all start from the same depot $0$ at time $0$, have the same capacity $W$, and must return to the depot withing $t_{max}$. Each vehicle can exit the depot at most once. The problem can be formulated on a complete directed graph $G = (V, A)$ where $V = C \cup \{0\}$ is the set of nodes and $A = \{(i,j) | i, j \in V, i \neq j\}$ is the set of arcs. Each arc can be traveled at most once by any of the vehicles. For each arc $(i, j) \in A$, let us define the time $t_{ij}$ required to travel over the arc. Travel times satisfy the triangle inequality. The goal of the company is to minimize the total time required to complete the service for all customers. Provide a mathematical formulation of the problem and its optimal solution. 

### Mathematical Model

## Import Libraries

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import gurobipy as gp
from gurobipy import GRB

## First point

In [63]:

# Sample data
N_0 = [1, 2, 3]  # Customers
K = [1,2]     # Vehicles
N = N_0 + [0]    # Nodes (including depot 0)
A = [(i, j) for i in N for j in N if i != j] #set of arcs
Q = 10         # Vehicle capacity
q = {1: 2, 2: 3, 3: 4}  # Package weights
t = {0: 0, 1: 40, 2: 20, 3: 10}  # Service times for each customer
# Travel times
t_ij = {
    (0, 1): 20, (0, 2): 20, (0, 3): 20,
    (1, 0): 20, (1, 2): 25, (1, 3): 30,
    (2, 0): 40, (2, 1): 25, (2, 3): 35,
    (3, 0): 30, (3, 1): 30, (3, 2): 35
}
a = {1: 5, 2: 15, 3: 25}  # lower time window constraint
b = {1: 100, 2: 100, 3: 100} # upper time window constraint
R = [(1,3,2)] # if client 1 and 2 are served by the same van then client 3's van must be different

In [64]:
model = gp.Model("last_mile_delivery_with_time_windows_and_restrictions")
x = model.addVars(A, vtype=GRB.BINARY, name="x") #1 if a vehicle traverses (i, j), 0 otherwise
f = model.addVars(A, vtype=GRB.INTEGER, name = "f") #amount of commodity flowing on arc (i,j) 
y = model.addVars(N_0, vtype=GRB.INTEGER, name = "y") #starting time of service at node j
s = model.addVars(N_0, vtype = GRB.INTEGER, name="s") #route duration if j is the last node visited before the depot
#z = model.addVars(N_0, K, vtype=GRB.BINARY, name="z") #1 if node i is served by van k

model.setObjective(gp.quicksum((t_ij[i, j]) * x[i, j] for (i, j) in A), GRB.MINIMIZE)


# (5) number of exit vehicles == total number of vehicles
constraint_vehicles = None
for j in N_0:
    if constraint_vehicles == None:
        constraint_vehicles = x[0, j]
    else:
        constraint_vehicles += x[0, j]
model.addConstr(constraint_vehicles == len(K))

# (6) Vehicles entering in each node == 1
for i in N_0:
    constraint_entering_flow = None
    for j in N:
        if i != j:
            if constraint_entering_flow == None:
                constraint_entering_flow = x[i, j]
            else:
                constraint_entering_flow += x[i, j]
    model.addConstr(constraint_entering_flow == 1)
# (7) Vehicles out of each node == 1
for j in N_0:
    constraint_exit_flow = None
    for i in N:
        if i != j:
            if constraint_exit_flow == None:
                constraint_exit_flow = x[i, j]
            else:
                constraint_exit_flow += x[i, j]
    model.addConstr(constraint_exit_flow == 1)

# (8) Control that each package is served correctly to each node
for i in N_0:
    constraint_quantity_stability = None
    sum_fji = None
    sum_fij = None
    for j in N:
        if i != j:
            if sum_fji == None:
                sum_fji = f[j, i]
            else:
                sum_fji += f[j, i]
            if sum_fij == None:
                sum_fij = f[i, j]
            else:
                sum_fij += f[i, j]
    model.addConstr((sum_fji - sum_fij) == q[i])

# (9) current capacity > next necessary capacity
for (i,j) in A:
    if j != 0:
        model.addConstr((q[j] * x[i, j]) <= f[i, j])
    if i != 0:
        model.addConstr(f[i, j] <= (Q - q[i])*x[i, j])

# (11) ???????????
M = 10e7
for (i, j) in A:
    if i != 0 and j != 0:
        model.addConstr(y[i] - y[j] + t[i] + t_ij[i, j] <= M*(1 - x[i, j]))
    elif j != 0:
        model.addConstr(-y[j] + t[i] + t_ij[i, j] <= M*(1 - x[i, j]))

L = 10e7
for j in N_0:
    model.addConstr(y[j] + t[j] - s[j]  <= L*(1 - x[j, 0]))

# (10) time window constraint
for i in N_0:
    model.addConstr(y[i] >= a[i], name=f"time_window_lower_{i}")
    model.addConstr(y[i] <= b[i], name=f"time_window_upper_{i}")

# tuple constraint with variable z (doesn't work)
'''
for i in N_0:
    model.addConstr(gp.quicksum(z[i, k] for k in K) == 1)

for i in N_0:
    for j in N_0:
        if i != j:
            for k in K:
                model.addConstr(x[i, j] <= z[i, k])
                model.addConstr(x[i, j] <= z[j, k])
        
for (i, j, l) in R:
    for k in K:
        model.addConstr(z[i, k] + z[j, k] <= 1 + z[l, k])
'''

# tuple constraint without variable z
# direct linkage between i,j,l
for (i, j, l) in R:
    model.addConstr(x[i, j] + x[i, l] + x[j, l] <= 1)

# undirect linkage between i,j,l (example: i ==> j ==> k ==> l) consider only one intermediate node
for (i, j, l) in R:
    for k in N_0:
        if k != i and k != j and k != l:
            model.addConstr(x[i, j] + x[j, k] + x[i, k] + x[k, l] <= 2)

In [65]:
model.optimize()

# Display results
if model.Status == GRB.OPTIMAL:
    solutions = model.getAttr('x', x)
    for key in solutions:
        if solutions[key] > 0.5:
            print(key, "->",solutions[key])
    model.write('model.lp')
    for v in model.getVars():
        if 'y' in  v.VarName :
            print(f"{v.VarName} = {v.X}")
        if 's' in v.VarName:
            print(f"{v.VarName} = {v.X}")
if model.Status == GRB.INFEASIBLE:
        model.computeIIS()
        model.write('model.ilp')

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 47 rows, 30 columns and 117 nonzeros
Model fingerprint: 0x20411127
Variable types: 0 continuous, 30 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective 125.0000000
Presolve removed 47 rows and 30 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 115 125 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.150000000000e+02, best bound 1.150000000000e+02, gap 0.0000%
(1, 0) -> 1.0
(2, 1) -> 1.0
(3, 0) -> 1.0
(0, 2) -> 1.0
(0, 3